In [12]:
import torch
from SSP.pruned import pruned_conv_layer
from torchvision.models.segmentation import deeplabv3_mobilenet_v3_large
from torch.utils.data import DataLoader
from torch import nn
from torch import optim
from torchvision.models.segmentation import deeplabv3_mobilenet_v3_large, DeepLabV3_MobileNet_V3_Large_Weights
from torchvision.datasets.voc import VOCSegmentation
from SSP.process_voc import VOCSegmentationWithJointTransform, JointTransform
from tqdm.auto import tqdm

/home/sam/Documents/perso/project/SSP/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = deeplabv3_mobilenet_v3_large(pretrained=True, num_classes=21)
model.load_state_dict(torch.load("model_weights.pth"), strict=True)
model.eval();

/home/sam/Documents/perso/project/SSP/venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/sam/Documents/perso/project/SSP/venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_MobileNet_V3_Large_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
amount = 0.4
pruned_conv_layer(model, amount)

In [10]:
torch.save(model.state_dict(), f"model_weights_pruned_{amount}.pth")


In [13]:
dataset = VOCSegmentationWithJointTransform(
    root='data',
    year='2012',
    image_set='train',
    download=True,
    joint_transform=JointTransform()
)

In [14]:
loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)
criterion = nn.CrossEntropyLoss(ignore_index=255)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 5
for epoch in tqdm(range(num_epochs)):
    for images, masks in loader:
        images, masks = images.to(device), masks.to(device)
        outputs = model(images)['out']
        loss = criterion(outputs, masks)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [06:49<00:00, 81.93s/it]


In [16]:
torch.save(model.state_dict(), f"F_model_weights_pruned_{amount}.pth")